In [ ]:
# if you want to have a pop up sign in method instead please uncomment  on ee.Authenticate and ee.Initialize, but comment on below

import ee

# Trigger the authentication flow. if you want to user json, please comment this
ee.Authenticate()
# Initialize the library.
ee.Initialize()

# here we try to use json api information instead (service account) - comment below if you use above methode
#import ee
import math
from datetime import datetime

# uncomment this if you want to use json instead
#service_account = 'iqbalpythonapi@bukit30project.iam.gserviceaccount.com'
#credentials = ee.ServiceAccountCredentials(service_account, './bukit30project-4d92e5b46ea7.json')
#ee.Initialize(credentials)

In [ ]:
FCD = ee.Image("users/muhfirdausiqbal/FCD_Check/FCD_LANDSATB30__2016_2017_no_TI_1_1")


In [ ]:
pca_scale = 30

In [ ]:
import geemap#.foliumap as geemap
import geopandas as gpd
import numpy as np
from sklearn.metrics import confusion_matrix

carbon_plot_xy = './shp_local/tnbt_plot_carbon.shp'
carbon_plot_xy_feat = geemap.shp_to_ee(carbon_plot_xy)

#band = high_forest_image.select([band_name_image])
    
# Unmask the band, replacing masked values (null) - Class: null with 0
unmasked_band = FCD.unmask(0)

# Use sampleRegions to extract pixel values at the point locations
sampled_points = unmasked_band.sampleRegions(
    collection=carbon_plot_xy_feat,
    scale=pca_scale,
    properties =['PLOT ID', 'Carbon Sto']
)

# Get the pixel values as a list
#pixel_values = sampled_points.aggregate_array('VD2').getInfo()


In [ ]:
sampled_points.getInfo()

In [ ]:
plot_arr = sampled_points.aggregate_array('PLOT ID').getInfo()

In [ ]:
plot_arr

In [ ]:
fcd_arr = sampled_points.aggregate_array('FCD').getInfo()

In [ ]:
carbon_stock_tcHa_arr = sampled_points.aggregate_array('Carbon Sto').getInfo()

In [ ]:
fcd_arr

In [ ]:
carbon_stock_tcHa_arr

In [ ]:
import pandas as pd

key_field = ['Plot_ID', 'FCD', 'Carbon_stock']
value_field = [plot_arr, fcd_arr, carbon_stock_tcHa_arr]

dict_data = {key_field[i]:value_field[i] for i in range(len(key_field)) }
data_pair_df = pd.DataFrame(dict_data)

In [ ]:
pd.set_option('display.max_rows', None)
data_pair_df

In [ ]:
# alternatively
gdf = gpd.read_file(carbon_plot_xy)

In [ ]:
gdf

In [ ]:
#gdf['Carbon Sto'].tolist()

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Check if fcd_arr and carbon_stock_tcHa_arr are loaded properly and have data
print("fcd_arr shape:", np.array(fcd_arr).shape)
print("carbon_stock_tcHa_arr shape:", np.array(carbon_stock_tcHa_arr).shape)

# Convert arrays to numpy arrays and ensure they are 1D
x = np.array(fcd_arr).reshape(-1)
y = np.array(carbon_stock_tcHa_arr)

# Check if there are any NaN or missing values in the arrays
print("NaN in x:", np.isnan(x).any())
print("NaN in y:", np.isnan(y).any())

# Create the LinearRegression model and perform fitting and predictions
model = LinearRegression()
model.fit(x.reshape(-1, 1), y)
y_pred = model.predict(x.reshape(-1, 1))

# Calculate R-squared score
r2 = r2_score(y, y_pred)
print("R-squared:", r2)

In [ ]:
# Create a DataFrame for comparison
comparison_df = pd.DataFrame({'Actual': y, 'Predicted': y_pred})

# Print the DataFrame
print(comparison_df)

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(x, y)
plt.xlabel('Forest Cover Density')
plt.ylabel('Carbon Stock')
plt.title('Scatter Plot: Forest Cover Density vs. Carbon Stock')
plt.show()

In [ ]:
x = x.reshape(-1)  # Remove any unnecessary dimensions
y = y.reshape(-1)

correlation = np.corrcoef(x, y)[0, 1]
print("Correlation coefficient:", correlation)

In [ ]:
y